# **Conversation Memory Management**
# **Introduction**
This notebook focuses on managing conversation memory effectively, including handling memory constraints, implementing memory policies, and managing conversation history.

## **Basic Memory Management**
**Let's start with basic conversation memory handling:**

In [128]:
from swarmauri.llms.concrete.GroqModel import GroqModel
from swarmauri.messages.concrete import SystemMessage, HumanMessage, AgentMessage
from swarmauri.conversations.concrete.SessionCacheConversation import SessionCacheConversation
import os
from dotenv import load_dotenv

**Initialize the Groq model**

In [129]:
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")
llm = GroqModel(api_key=api_key)

 **LLM basic memory Setup:**
 Create a conversation with limited memory
 

 

In [130]:
# Create conversation with memory limit
conversation = SessionCacheConversation(
    system_context=SystemMessage(content="You are a knowledgeable research assistant"),
    max_size=2,session_max_size=5
)

 **Add  conversations**

In [131]:
# Test question series
questions = [
    "in  a sentence Tell me about renewable energy",
    "in  a sentence What about solar power specifically?",
    "in  a sentence How do solar panels work?",
    "in  a sentence What's their efficiency?"
]

In [133]:
print("Responses:")
for question in questions:
    conversation.add_message(HumanMessage(content=question))
    response = llm.predict(conversation)
    conversation.add_message(AgentMessage(content=response.get_last().content))
    print(f"\nQ: {question}")
    print(f"A: {response.get_last().content}")

Responses:

Q: in  a sentence Tell me about renewable energy
A: Renewable energy sources like solar, wind, hydro, geothermal, and tidal energy harness naturally replenished resources to generate electricity, heat homes, and power transportation, offering sustainable and environmentally friendly alternatives to fossil fuels.

Q: in  a sentence What about solar power specifically?
A: Solar power harnesses the abundant energy of the sun through photovoltaic cells or thermal collectors, generating electricity or heat for various applications.

Q: in  a sentence How do solar panels work?
A: Solar panels harness the energy of the sun through the photovoltaic effect. Sunlight is absorbed by silicon atoms in the solar cells, causing electrons to be excited and creating a flow of electricity.

Q: in  a sentence What's their efficiency?
A: Their efficiency varies depending on the specific task, with some models achieving human-parity in certain areas like language translation and question-answer

In [135]:
for msg in conversation.history:
       print(msg.content)
       

You are a knowledgeable research assistant


In [137]:
for msg in conversation.session:
       print(msg.content)

Solar panels harness the energy of the sun through the photovoltaic effect. Sunlight is absorbed by silicon atoms in the solar cells, causing electrons to be excited and creating a flow of electricity.
Solar panels harness the energy of the sun through the photovoltaic effect. Sunlight is absorbed by silicon atoms in the solar cells, causing electrons to be excited and creating a flow of electricity.
in  a sentence What's their efficiency?
Their efficiency varies depending on the specific task, with some models achieving human-parity in certain areas like language translation and question-answering, while others struggle with tasks that require common sense or physical interaction.
Their efficiency varies depending on the specific task, with some models achieving human-parity in certain areas like language translation and question-answering, while others struggle with tasks that require common sense or physical interaction.


## **Differentiating max_size and session_cache_max_size**



This example demonstrates how max_size and session_cache_max_size work differently. 
The max_size parameter determines how many recent messages the model is aware of, while session_cache_max_size controls how many messages the user can see in the conversation history.

**Create conversation with system context**

In [91]:


conversation = SessionCacheConversation(
    system_context=SystemMessage(content="You are a helpful cooking assistant"),
    max_size=3,             # Keeps last 3 exchanges in memory
    session_max_size=10   # Keeps last 6 exchanges in session cache
)


**Simulate a chat with cooking questions**

In [92]:

cooking_exchanges = [
    ("How do I make pasta carbonara?", "To make pasta carbonara, you'll need..."),
    ("What ingredients do I need?", "For the ingredients, you'll want eggs, pancetta..."),
    ("How long should I cook the pasta?", "Cook the pasta for about 8-10 minutes."),
    ("How do I know when it's done?", "The pasta is done when it's tender."),
    ("Should I add salt?", "Yes, add a pinch of salt to the water."),
    ("Can I add cream?", "Traditionally, carbonara doesn’t use cream.")
]

**Add each message in the conversation**

In [93]:

for question, response in cooking_exchanges:
    conversation.add_messages([
        HumanMessage(content=question),
        AgentMessage(content=response)
    ])


**Show conversation history (affected by max_size)**

In [100]:

print("\nCONVERSATION HISTORY (max_size=3):")
for message in conversation.history:
    print(f"\n{message.content}")


CONVERSATION HISTORY (max_size=3):

You are a helpful cooking assistant

Can I add cream?

Traditionally, carbonara doesn’t use cream.


**Show session cache (affected by session_cache_size)**

In [101]:

print("\nSESSION CACHE (session_cache_size=6):")
for message in conversation.session:
    print(f"\n{message.content}")


SESSION CACHE (session_cache_size=6):

What ingredients do I need?

For the ingredients, you'll want eggs, pancetta...

How long should I cook the pasta?

Cook the pasta for about 8-10 minutes.

How do I know when it's done?

The pasta is done when it's tender.

Should I add salt?

Yes, add a pinch of salt to the water.

Can I add cream?

Traditionally, carbonara doesn’t use cream.


## **Differentiating max_size and session_cache_max_size with LLM**

In the example, Conversation 1 has a max_size of 3 and a session_cache_max_size of 10, while Conversation 2 has a max_size of 6 and a session_cache_max_size of 20. 
This means that the model in Conversation 1 is only aware of the last 3 messages, but the user can see up to 10 messages in the history. Similarly, the model in Conversation 2 is aware of the last 6 messages, but the user can see up to 20 messages.


**This is helpful as it keeps token cost low but still allow the you to be able to recall previous parts of the conversation**

**Create conversations with different memory limits:**

Conversation1 has a short model memory with short user chat history


In [33]:
# max_size sets the model's awareness of recent messages
# session_cache_max_size sets the number of messages the user can see
conversation1 = SessionCacheConversation(
    system_context=SystemMessage(content="You are a research assistant"),
    max_size=2,session_max_size=4
)


Conversation2  has a long model memory with short user chat history

In [102]:
conversation2 = SessionCacheConversation(
    system_context=SystemMessage(content="You are a technical support agent"), 
    max_size=6,
    session_max_size=10
)

**Add messages to both conversations**

In [103]:
# Add messages to both conversations
research_questions = [
    "in  a sentence What is machine learning?",
    "in  a sentence How does it work?", 
    "in  a sentence What are the main applications?",
    "in  a sentence What are the challenges?",
    "in  a sentence What is a list in Python?",
    "in  a sentence How do I create a dictionary?",
    "in  a sentence What are list comprehensions?"
]

for question in research_questions:
    conversation1.add_message(HumanMessage(content=question))
    conversation1.add_message(AgentMessage(content=llm.predict(conversation1).get_last().content))

for question in research_questions:
    conversation2.add_message(HumanMessage(content=question))
    conversation2.add_message(AgentMessage(content=llm.predict(conversation2).get_last().content))


**Print conversation1 model memory and session cache(number of messages users can see)**

Model memory and awareness

In [109]:
# Print the list of information retained in the model memory
print("Conversation 1 (max_size=2, session_cache_max_size=4):")
for msg in conversation1.history:
    print (msg.content)


Conversation 1 (max_size=2, session_cache_max_size=4):
You are a research assistant
in  a sentence What are list comprehensions?
List comprehensions are a syntax in some programming languages that allows creating new lists by iterating over existing lists and applying a transformation or condition to each element.


Session cache (number of messages users can see)

In [110]:
# Print the number of messages users can see
print("Conversation 1 (max_size=2, session_cache_max_size=4):")
for msg in conversation1.session:
    print (msg.content)

Conversation 1 (max_size=2, session_cache_max_size=4):
There are numerous challenges associated with the research process, including limited access to data, methodological limitations, and biases in data collection and analysis.
in  a sentence What is a list in Python?
A list in Python is a mutable, ordered collection of elements that can hold various data types like integers, strings, floats, or even other lists. It is denoted by square brackets `[]` and can contain multiple values separated by commas.
A list in Python is a mutable, ordered collection of elements that can hold various data types like integers, strings, floats, or even other lists. It is denoted by square brackets `[]` and can contain multiple values separated by commas.
in  a sentence How do I create a dictionary?
Creating a dictionary involves meticulous planning, data collection, and meticulous organization. You'll need to define the scope, determine the language or domain, gather and analyze data, establish indexin

**Print Conversation2 model memory and session cache(number of messages users can see)**

Model memory and awareness

In [115]:

print("\nCONVERSATION 2 (max_size=6, session_cache_max_size=10):")
for msg in conversation2.history:
    print(msg.content)


CONVERSATION 2 (max_size=6, session_cache_max_size=10):
You are a technical support agent
in  a sentence How do I create a dictionary?
You can create a dictionary in Python using curly braces { } and pairing keys with values. Keys must be unique and of a immutable type, while values can be of any type. For example, `my_dict = {"name": "John", "age": 30}` creates a dictionary with the key-value pairs "name" -> "John" and "age" -> 30.
in  a sentence What are list comprehensions?
List comprehensions are a concise and efficient way to create new lists by iterating over existing lists and performing transformations or conditional logic on each element.


Session cache (number of messages users can see)

In [116]:

print("\nCONVERSATION 2 (max_size=6, session_cache_max_size=10):")
for msg in conversation2.session:
    print(msg.content)


CONVERSATION 2 (max_size=6, session_cache_max_size=10):
Machine learning systems can suffer from biases in the training data, leading to fairness and privacy concerns, and their performance can degrade in real-world environments due to unforeseen circumstances.
in  a sentence What is a list in Python?
A list in Python is an ordered collection of elements that can be of any type, allowing for flexible data manipulation and retrieval.
A list in Python is an ordered collection of elements that can be of any type, allowing for flexible data manipulation and retrieval.
in  a sentence How do I create a dictionary?
You can create a dictionary in Python using curly braces { } and pairing keys with values. Keys must be unique and of a immutable type, while values can be of any type. For example, `my_dict = {"name": "John", "age": 30}` creates a dictionary with the key-value pairs "name" -> "John" and "age" -> 30.
You can create a dictionary in Python using curly braces { } and pairing keys wit

In [112]:
print("\nConversation 1 model's memory size:", len(conversation1.history))
print("Conversation 2 model's memory size:", len(conversation2.history))


Conversation 1 model's memory size: 3
Conversation 2 model's memory size: 5


# **Conclusion**
**In this notebook, we've explored:**



Integration of Groq LLM with memory management

Handling memory constraints in LLM conversations

Comparing different memory policies with LLM responses

Managing complex conversations with memory tracking

Understanding how memory affects LLM context and responses

**These techniques help create more effective and memory-efficient conversational AI applications.**

## **NOTEBOOK METADATA**

In [26]:
import os
import platform
import sys
from datetime import datetime

# Display author information
author_name = "Dominion John " 
github_username = "DOMINION-JOHN1"  

print(f"Author: {author_name}")
print(f"GitHub Username: {github_username}")

# Last modified datetime (file's metadata)
notebook_file = "Notebook_04_Conversation_Memory.ipynb"
try:
    last_modified_time = os.path.getmtime(notebook_file)
    last_modified_datetime = datetime.fromtimestamp(last_modified_time)
    print(f"Last Modified: {last_modified_datetime}")
except Exception as e:
    print(f"Could not retrieve last modified datetime: {e}")

# Display platform, Python version, and Swarmauri version
print(f"Platform: {platform.system()} {platform.release()}")
print(f"Python Version: {sys.version}")

import swarmauri

try:
    version = swarmauri.__version__
except AttributeError:
    version = f"Swarmauri Version: 0.5.1"

print(f"Swarmauri Version: {version}")

Author: Dominion John 
GitHub Username: DOMINION-JOHN1
Last Modified: 2024-11-04 13:33:22.996990
Platform: Windows 11
Python Version: 3.12.7 (tags/v3.12.7:0b05ead, Oct  1 2024, 03:06:41) [MSC v.1941 64 bit (AMD64)]
Swarmauri Version: Swarmauri Version: 0.5.1
